In [1]:
!pip install transformers sentencepiece langchain faiss-cpu

!pip install pypdf unstructured bs4

  Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)
  Using cached sentencepiece-0.2.0-cp39-cp39-win_amd64.whl (991 kB)
  Using cached langchain-0.3.26-py3-none-any.whl (1.0 MB)
  Using cached faiss_cpu-1.11.0.post1-cp39-cp39-win_amd64.whl (14.9 MB)
  Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl (274 kB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
  Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl (2.5 MB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached huggingface_hub-0.33.4-py3-none-any.whl (515 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached langchain_core-0.3.69-py3-none-any.whl (441 kB)
  U

You should consider upgrading via the 'C:\Users\ASUS\Desktop\vpbank_hackathon\models_venv\Scripts\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 309.7/309.7 KB 2.1 MB/s eta 0:00:00
  Using cached unstructured-0.18.3-py3-none-any.whl (1.8 MB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
  Using cached unstructured_client-0.38.1-py3-none-any.whl (212 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Using cached python_oxmsg-0.0.2-py3-none-any.whl (31 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached lxml-6.0.0-cp39-cp39-win_amd64.whl (4.0 MB)
  Using cached wrapt-1.17.2-cp39-cp39-win_amd64.whl (38 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached emoji-2.14.1-py3-none-any.whl (590 kB)
  Us

You should consider upgrading via the 'C:\Users\ASUS\Desktop\vpbank_hackathon\models_venv\Scripts\python.exe -m pip install --upgrade pip' command.


# vinai/phobert-base


In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

from langchain.embeddings.base import Embeddings
from transformers import AutoTokenizer, AutoModel
import torch

class PhoBERTEmbeddings(Embeddings):
    def __init__(self, model_name="vinai/phobert-base"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()
        self.model.to("cpu")

    def embed_documents(self, texts):
        return [self._get_cls_embedding(text) for text in texts]

    def embed_query(self, text):
        return self._get_cls_embedding(text)

    def _get_cls_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        cls_embedding = outputs.last_hidden_state[0][0]
        return cls_embedding.cpu().tolist()


c:\Users\ASUS\Desktop\vpbank_hackathon\models_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
phobert_embedder = PhoBERTEmbeddings()

# Tải docs / tạo vector , lưu vector



In [ ]:
import os
from langchain.document_loaders import PyPDFLoader

def load_all_pdfs(folder_path):
    all_docs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(folder_path, filename))
            docs = loader.load()
            all_docs.extend(docs)
    return all_docs

documents = load_all_pdfs("Vpbank_docs/pdf_docs")


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.split_documents(documents)


In [ ]:
from langchain.vectorstores import FAISS

phobert_embedder = PhoBERTEmbeddings()
vectorstore = FAISS.from_documents(docs, phobert_embedder)

In [ ]:
vectorstore.save_local("phobert_faiss_index")

# truy vấn

In [10]:
from langchain.vectorstores import FAISS

# Khởi tạo lại PhoBERTEmbeddings (nếu chưa)
phobert_embedder = PhoBERTEmbeddings()

# Load lại FAISS index đã lưu
vectorstore = FAISS.load_local(
    folder_path="phobert_faiss_index",
    embeddings=phobert_embedder,
    allow_dangerous_deserialization=True  # Bắt buộc khi không dùng OpenAI
)


In [ ]:
# Cách ổn định và dễ đọc nhất
def print_docs(docs):
    for i, doc in enumerate(docs):
        print(f"\n🔹 Kết quả {i+1}")
        filename = doc.metadata['source'].split("Vpbank_docs\\")[-1]  # hoặc dùng os.path.basename
        page = doc.metadata.get("page_label", doc.metadata.get("page", "?"))
        print(f"📄 Tệp: {filename}")
        print(f"🔖 Trang: {page}")
        print(f"📝 Nội dung:\n{doc.page_content.strip()}")


In [12]:
import numpy as np

query_vec = phobert_embedder.embed_query("Tôi muốn biết thủ tục mở thẻ tín dụng")
query_vec = np.array(query_vec, dtype=np.float32)  # ép kiểu bắt buộc

results = vectorstore.similarity_search_by_vector(query_vec)
print_docs(results)


🔹 Kết quả 1
📄 Tệp: 4pl02bieuphikhuutienvertiengvietcapnhat21112024docxhi4mq.pdf
🔖 Trang: 21
📝 Nội dung:
 Ủy quyền Thẻ tiết kiệm 
 Phong tỏa tiền gửi tiết kiệm theo yêu cầu của KH 
 Xác nhận số dư/thông tin tiền gửi tiết kiệm 
 Lập giấy ủy quyền tại VPBank để thực hiện giao dịch đối với tiền gửi tiết kiệm/giấy tờ có giá  
 Xử lý rút tiền theo hồ sơ thừa kế

🔹 Kết quả 2
📄 Tệp: bieu-phi-dich-vu-khach-hang-ca-nhan-hieu-luc-tu-01-09-2024.pdf
🔖 Trang: 16
📝 Nội dung:
- Biểu phí này chưa bao gồm VAT. 
- Số dư bình quân (SDBQ) TKTT tại Biểu phí này được xác định theo công thức sau và có thể được VPBank 
thay đổi từng thời kỳ theo quy định của VPBank: Số dư bình quân bằng (=) Tổng số dư cuối ngày của

🔹 Kết quả 3
📄 Tệp: 4pl02bieuphikhuutienvertiengvietcapnhat21112024docxhi4mq.pdf
🔖 Trang: 6
📝 Nội dung:
Xác nhận tài khoản theo yêu cầu 
của chủ thẻ 50,000VND 50,000VND 50,000VND 50,000VND Không thu phí Không thu phí

🔹 Kết quả 4
📄 Tệp: bieu-phi-dich-vu-khach-hang-ca-nhan-hieu-luc-tu-01-09-202